### From google colab

In [ ]:
# "/content/drive/MyDrive/Investimentos/JA-Investimentos"
!pip install tabula-py
!apt-get install openjdk-8-jre-headless -qq > /dev/null

In [ ]:
import pandas as pd
import tabula
import os

In [ ]:
path_pdf = "/content/drive/MyDrive/Investimentos/JA-Investimentos"
tabelas = []
for arquivo in os.listdir(path_pdf):
    if arquivo.endswith('.pdf'):
        # Extrair tabelas da primeira página do arquivo PDF
        try:
            df = tabula.read_pdf(os.path.join(path_pdf, arquivo), pages=1)[0]
            df.columns=["Data","Quantidade apostas", "Valor investido", "Resultado banca"]
            df.drop(df.index[-1], inplace=True)
            tabelas.append(df)
        except Exception as e:
            print(f"Erro ao processar o arquivo {arquivo}: {e}")

In [ ]:
df = pd.concat(tabelas, ignore_index=True)

In [ ]:
df.dropna(how='any', axis=0, inplace=True)

In [ ]:
df.columns=["Data","Quantidade apostas", "Valor investido", "Resultado banca"]
df[['Invest', 'Retorno']] = df["Valor investido"].str.split(r'\s*\/\s*', expand=True) #separo minha coluna
df.drop(columns=['Valor investido'], inplace=True) #Removo a coluna que eu separei anteriormente.
df = df[df['Quantidade apostas'] != 'Sem apostas']
df.head()
df.describe()

In [ ]:
try:
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
    df['Invest'] = df['Invest'].str.replace(r'[^\d.-]', '', regex=True).astype(float)
    df['Retorno'] = df['Retorno'].str.replace(r'[^\d.-]', '', regex=True).astype(float)
    df['Resultado banca'] = df['Resultado banca'].str.replace(' ', '', regex=True).str.replace('--', '-', regex=True)
    # Remover o '%' e todos os caracteres não numéricos, exceto o '.' para preservar o ponto decimal
    df['Resultado banca'] = df['Resultado banca'].str.replace('%', '').str.replace(r'[^\d.-]', '', regex=True)
    # Substituir múltiplos pontos decimais por apenas um
    df['Resultado banca'] = df['Resultado banca'].str.replace(r'\.{2,}', '.', regex=True)
    # Substituir a vírgula por ponto como separador decimal
    df['Resultado banca'] = df['Resultado banca'].str.replace(',', '.')
    # Converter para float
    df['Resultado banca'] = df['Resultado banca'].astype(float)
    df['Quantidade apostas'] = df['Quantidade apostas'].astype(int)
except ValueError as errorCod:
    print("Unable to convert a value: ", errorCod)
display(df)

In [ ]:
df['Invest'] = df['Invest'] / 100
df['Resultado banca'] = df['Resultado banca'] / 10000
df['Retorno'] = df['Retorno'] / 100
df.head()

In [ ]:
df.info()
df.describe()

In [ ]:
df.to_csv('/content/drive/MyDrive/Investimentos/JA-Investimentos/csv/ate_11-03-24.csv', index=False)